In [67]:
from pathlib import Path
from pprint import pprint
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import spacy

pd.options.display.max_colwidth = 60

nlp = spacy.load('en', disable=["tagger", "parser", "ner"])

In [55]:
wd = Path('/home/mquezada/CrisisMMD/annotations/california_wildfires_final_data.tsv')

In [56]:
df = pd.read_table(wd)
df.head(3)

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path
0,917791044158185473,917791044158185473_0,informative,1.0000,informative,0.6766,other_relevant_information,1.0000,other_relevant_information,0.6766,NaN,NaN,RT @Gizmodo: Wildfires raging through Northern Californi...,http://pbs.twimg.com/media/DLyi_WYVYAApwNg.jpg,data_image/california_wildfires/10_10_2017/9177910441581...
1,917791130590183424,917791130590183424_0,informative,1.0000,informative,0.6667,infrastructure_and_utility_damage,1.0000,affected_individuals,0.6667,NaN,NaN,PHOTOS: Deadly wildfires rage in California https://t.co...,http://pbs.twimg.com/media/DLymKm9UMAAu0qw.jpg,data_image/california_wildfires/10_10_2017/9177911305901...
2,917791291823591425,917791291823591425_0,informative,0.6813,informative,1.0000,other_relevant_information,0.6813,infrastructure_and_utility_damage,1.0000,severe_damage,1.0,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wildfire respo...,http://pbs.twimg.com/media/DLudaaZV4AAjT7x.jpg,data_image/california_wildfires/10_10_2017/9177912918235...


In [57]:
df.shape

(1589, 15)

In [18]:
texts = df['tweet_text']
text_l = texts.tolist()
print(text_l[:3])

['RT @Gizmodo: Wildfires raging through Northern California are terrifying https://t.co/dI73RFzX2i https://t.co/k4KnvIimsU', 'PHOTOS: Deadly wildfires rage in California https://t.co/td9xT3vXOL https://t.co/OimwAncLew', 'RT @Cal_OES: PLS SHARE: Weâ€™re capturing wildfire response, recovery info here: https://t.co/r89LKpjLPj https://t.co/HiA1oQF2Ax']


In [44]:
#### extracts urls from texts

wd = Path('/home/mquezada/CrisisMMD/annotations/')
for f in wd.glob('*.tsv'):
    df = pd.read_table(f)
    texts_l = list(df['tweet_text'])
    with open(f'/home/mquezada/news-model-git/news-model/data_crisismmd/{f.name}_urls.txt', 'w') as f:
        for text in nlp.pipe(text_l, n_threads=16):
            for token in text:
                f.write(token.text + "\n") if token.like_url else None

In [45]:
#### load expanded urls

expanded_urls = dict()
wd = Path('/home/mquezada/news-model-git/news-model/data_crisismmd/expanded_urls.tsv')
with wd.open() as f:
    for line in f:
        short, exp = line.split('\t')
        expanded_urls[short] = exp[:-1]


In [48]:
# missing urls
missing = 0
for s, e in expanded_urls.items():
    if e == 'None':
        missing += 1

print("missing", missing)
print("total", len(expanded_urls))

missing 273
total 2718


In [51]:
# algunos tweets están repetidos
df['tweet_id'].value_counts()

912275136212013056    4
913559017741602816    4
927982073717157891    4
921298851687940096    4
911983033477140480    4
912346586910425088    4
915319640145911809    4
911650692368158721    4
921316214441463808    4
922137652534611969    4
912389558016323584    4
915973468201586689    4
913229700771844096    4
923787393471209472    4
922765857276792838    4
923228294618779648    4
912204795427123200    4
922520581727285250    4
921805309051535361    4
911648744675860480    4
913126475917803520    4
913934298243305473    4
922420405545066496    4
923329426158505985    4
914871738722390016    4
914086081666019328    4
910545522133528576    4
910523398253461504    4
921304553840152576    4
914221296904544256    4
                     ..
910792736453922816    1
927706645051904000    1
912996928568397825    1
912446485186441218    1
913045010723074048    1
922052175743688706    1
910582078470852608    1
912130173541273600    1
911549679229390859    1
916139079032541184    1
9287547446629130

In [59]:
df_uniq = df.drop_duplicates(subset='tweet_id')
df_uniq.shape

(1486, 15)

In [90]:
documents = defaultdict(list)
wo_url = []

for idx, t in tqdm(df_uniq.iterrows()):
    t_id = t['tweet_id']
    text = t['tweet_text']
    tokens = nlp(text)
    added = False
    for token in tokens:
        if token.like_url:
            expanded_url = expanded_urls.get(token.text)
            if expanded_url and expanded_url != 'None':
                documents[expanded_url].append(t_id)
                added = True
    if not added:
        wo_url.append(t_id)

1486it [00:00, 3471.06it/s]


In [100]:
for url, ids in sorted(documents.items(), key=lambda r: len(r[1]), reverse=True):
    print(len(ids), url, sep='\t')

85	https://twitter.com/account/suspended
21	http://www.mambolook.com/california/fire-accidents
9	http://www.mambolook.com/california/fire-accidents/fire-victims
6	http://www.project.nsearch.com/video/60-geoengineered-fire-storms-in-northern-california-out-of-1
3	https://t.co/
3	http://ussanews.com
3	http://www.mambolook.com/california
3	https://www.breitbart.com/california/2017/10/20/bannon-fire-ca-three-standing-ovations-gop-convention-keynote/
2	https://www.washingtonpost.com/news/post-nation/wp/2017/10/10/pure-devastation-at-least-15-dead-as-firefighters-struggle-to-weaken-california-fires/
2	http://www.mambolook.com/natural-disasters/wildfires
2	http://www.mambolook.com/idaho
2	https://www.theguardian.com/world/2017/oct/15/california-wildfires-death-toll-evacuations?utm_source=dlvr.it&utm_medium=twitter
2	https://viralhug.com
2	https://www.wired.com/story/photo-of-the-week-california-wildfire/
2	https://www.nytimes.com/interactive/2017/10/14/us/california-fire-building-damage.html


1	https://www.youtube.com/watch?v=-pIt5N97ihg
1	https://twitter.com/jmnese/status/920997740972265472/photo/1
1	https://www.mysanantonio.com/bayarea/article/Tubbs-Fire-unleashed-fiery-tornadoes-that-12289228.php
1	https://twitter.com/rebeccalsalinas/status/920998722212950016/photo/1
1	https://www.shrm.org/resourcesandtools/hr-topics/employee-relations/pages/california-wildfires.aspx?utm_content=buffer625eb&utm_medium=social&utm_source=twitter.com&utm_campaign=buffer
1	https://twitter.com/SHRMHRNews/status/921006774953676801/photo/1
1	https://www.cbc.ca/news/world/california-wildfires-victims-eldery-couples-1.4361765?cmp=rss
1	https://twitter.com/CBCWorldNews/status/921009874372366336/photo/1
1	https://twitter.com/USSANews/status/921011331855036418/photo/1
1	https://prideradioaustin.iheart.com/content/2017-10-19-californias-other-big-problem-amid-blazes-fire-devils/
1	https://twitter.com/prideradiotx/status/921018323403526144
1	https://twitter.com/CookeCountyLib/status/921020713619853312

In [89]:
df_uniq.loc[df_uniq['tweet_id'] == 919975227953762304]['tweet_text']

794    California couple hid in pool as deadly wildfires swallo...
Name: tweet_text, dtype: object

In [101]:
df_uniq['tweet_text'].value_counts()

Fire at Large California Oil RefineryÂ Doused https://t.co/4RjIttKUUb https://t.co/DpOBah2ar4                                                   1
#Arson by nuts trying to prove #Gore was right? https://t.co/synPbrxJTE https://t.co/nW1iuoXWpA                                                 1
#MountVernonWA fire crew helping fight California wildfires https://t.co/6HJ6obpKyN #Skagit https://t.co/FAU0ZJZe4f                             1
DISASTER - #CALIFORNIA FIRE â€“ HOW TO GET HELP - How to get help after a fire https://t.co/XcqeK0Xkid https://t.co/FtxktqhXgc                  1
Family mourns Idaho man killed in California wildfires https://t.co/WLJvMvq5C4 https://t.co/u2Xv3YcNWy                                          1
Henry Cejudo suffers burns, loses gold medal in California wildfire â€” https://t.co/GkuLc60ZWl https://t.co/0gRZgSK4LU                         1
Female firefighter recruit sues California city for sexual harassment, retaliation https://t.co/NKE2US63Hs #fire https://t.c

In [116]:
import json

tweets_full = []

wd = Path('/home/mquezada/CrisisMMD/json/mexico_earthquake_final_data.json')
with wd.open() as f:
    for line in f:
        obj = json.loads(line)
        tweets_full.append(obj)

JSONDecodeError: Extra data: line 1 column 2992 (char 2991)

In [115]:
retweets = 0
replies = 0
for tweet in tweets_full:
    if tweet['retweeted']:
        retweets += 1
    if tweet['in_reply_to_status_id']:
        replies += 1
        
retweets, replies

(0, 20)

In [106]:
tweets_full[0]

{'filter_level': 'low',
 'quote_count': 0,
 'retweeted': False,
 'in_reply_to_screen_name': None,
 'possibly_sensitive': False,
 'truncated': False,
 'lang': 'en',
 'in_reply_to_status_id_str': None,
 'id': 917791130590183424,
 'extended_entities': {'media': [{'sizes': {'small': {'w': 680,
      'resize': 'fit',
      'h': 384},
     'thumb': {'w': 150, 'resize': 'crop', 'h': 150},
     'medium': {'w': 1200, 'resize': 'fit', 'h': 677},
     'large': {'w': 1200, 'resize': 'fit', 'h': 677}},
    'id': 917791125263364096,
    'media_url_https': 'https://pbs.twimg.com/media/DLymKm9UMAAu0qw.jpg',
    'media_url': 'http://pbs.twimg.com/media/DLymKm9UMAAu0qw.jpg',
    'expanded_url': 'https://twitter.com/WMTWTV/status/917791130590183424/photo/1',
    'indices': [68, 91],
    'id_str': '917791125263364096',
    'type': 'photo',
    'display_url': 'pic.twitter.com/OimwAncLew',
    'url': 'https://t.co/OimwAncLew'}]},
 'in_reply_to_user_id_str': None,
 'timestamp_ms': '1507653431924',
 'in_reply